In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

from utils.subspace_clustering_helper_funcs import *
from utils.ml_pipeline import *

In [2]:
import warnings
warnings.filterwarnings('ignore')
# C:\Users\kdmen\miniconda3\envs\fl_torch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning warnings.warn(

# Load in Data

In [1]:
# This requires loading in different data, according to 010a/b
if False:
    y_integer = np.argmax(encoded_labels_df, axis=1)
    #print(len(y_integer))
    ytest_integer = np.argmax(encoded_test_labels_df, axis=1)
    #print(len(ytest_integer))
    
    # CREATE THE TRAINING SET
    num_gestures = len(emg_training_users_df) // num_rows_per_gesture
    num_features = emg_training_users_df.shape[1]
    emg_train_loader = create_data_loader(emg_training_users_df, GestureDatasetClustering, labels=y_integer)
    # CREATE THE TEST SET
    num_features = emg_test_users_df.shape[1]
    num_gestures = len(emg_test_users_df) // num_rows_per_gesture
    emg_test_loader = create_data_loader(emg_test_users_df, GestureDatasetClustering, labels=ytest_integer, shuffle_bool=False)
    
    # CREATE THE TRAINING SET
    num_gestures = len(both_training_users_df) // num_rows_per_gesture
    num_features = both_training_users_df.shape[1]
    both_train_loader = create_data_loader(both_training_users_df, GestureDatasetClustering, labels=y_integer)
    # CREATE THE TEST SET
    num_features = both_test_users_df.shape[1]
    num_gestures = len(both_test_users_df) // num_rows_per_gesture
    both_test_loader = create_data_loader(both_test_users_df, GestureDatasetClustering, labels=ytest_integer, shuffle_bool=False)
    
    # CREATE THE TRAINING SET
    num_gestures = len(both_pca_training_users_df) // num_rows_per_gesture
    num_features = both_pca_training_users_df.shape[1]
    both_pca_train_loader = create_data_loader(both_pca_training_users_df, GestureDatasetClustering, labels=y_integer)
    # CREATE THE TEST SET
    num_test_features = both_pca_test_users_df.shape[1]
    num_test_gestures = len(both_pca_test_users_df) // num_rows_per_gesture
    both_pca_test_loader = create_data_loader(both_pca_test_users_df, GestureDatasetClustering, labels=ytest_integer, shuffle_bool=False)

In [4]:
# Kai's laptop
#data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\PCA_40D\\"
#data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\Filtered_Datasets\\"
#model_dir_path = 'C:\\Users\\kdmen\\Desktop\\Research\\Repos\\fl-gestures\\models\\Embedding\\Autoencoders\\'

# BRC Desktop
data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\Both_PPD\\"
full_data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\Filtered_Datasets\\"
embedding_data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\Autoencoders\\EMG_Only\\"
model_dir_path = 'C:\\Users\\YamagamiLab\\Desktop\\Dev\\fl-gestures\\models\\Embedding\\Autoencoders\\'

print("Loading")

metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']

both_full_train_XY_df = pd.read_pickle(data_path+'training_users_df.pkl')
both_full_test_XY_df = pd.read_pickle(data_path+'test_users_df.pkl')
both_full_gesture_train_labels_df = both_full_train_XY_df['Gesture_ID']
both_full_gesture_test_labels_df = both_full_test_XY_df['Gesture_ID']
both_full_user_train_labels_df = both_full_train_XY_df['Participant']
both_full_user_test_labels_df = both_full_test_XY_df['Participant']
both_full_training_u_df = both_full_train_XY_df.drop(metadata_cols, axis=1)
both_full_test_users_df = both_full_test_XY_df.drop(metadata_cols, axis=1)


Loading


One-hot encode the gestures so they can be used as labels. Should probably also do this for participant ID to use as a different label set but I'll do it later

In [5]:
# Get list of all the unique gesture names
all_train_test_gestures_df = pd.concat([full_pca40_gesture_train_labels_df, full_pca40_gesture_test_labels_df], axis=0)
unique_gestures = all_train_test_gestures_df.unique()

# Create OneHotEncoder instance
gesture_encoder = OneHotEncoder(sparse=False)
gesture_labels_reshaped = unique_gestures.reshape(-1, 1)
gesture_encoded = gesture_encoder.fit_transform(gesture_labels_reshaped)

In [6]:
print(full_pca40_train_XY_df.shape)
full_pca40_train_XY_df.head()

(160000, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [7]:
print(full_pca40_gesture_train_labels_df.shape)
full_pca40_gesture_train_labels_df.head()

(160000,)


0    pan
1    pan
2    pan
3    pan
4    pan
Name: Gesture_ID, dtype: object

In [8]:
print(full_pca40_training_u_df.shape)
full_pca40_training_u_df.head()

(160000, 88)


,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,IMU2_ay,IMU2_az,IMU2_vx,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,-0.883268,-0.327978,-0.221822,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,-0.944195,-0.363602,-0.149833,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,-0.944029,-0.357476,-0.164447,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,-0.911993,-0.335594,-0.006214,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,-0.939228,-0.357917,0.009543,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [10]:
num_rows_per_gesture = 64 # From the interp

# CREATE THE TRAINING SET
num_gestures = len(full_pca40_training_u_df) // num_rows_per_gesture
num_features = full_pca40_training_u_df.shape[1]
X_3D_PCA40 = full_pca40_training_u_df.to_numpy().reshape(num_gestures, num_rows_per_gesture, num_features)
X_3DTensor_PCA40 = torch.tensor(X_3D_PCA40, dtype=torch.float32)

# CREATE THE TEST SET
num_test_gestures = len(full_pca40_test_users_df) // num_rows_per_gesture
Xtest_3DTensor_PCA40 = torch.tensor(full_pca40_test_users_df.to_numpy().reshape(num_test_gestures, num_rows_per_gesture, num_features), dtype=torch.float32)


In [11]:
X_3DTensor_PCA40.shape

torch.Size([2500, 64, 88])

In [12]:
print(full_pca40_gesture_train_labels_df.shape)
full_pca40_gesture_train_labels_df.head()

(160000,)


0    pan
1    pan
2    pan
3    pan
4    pan
Name: Gesture_ID, dtype: object

In [13]:
label_per_gesture_df = full_pca40_gesture_train_labels_df.iloc[::64]
print(label_per_gesture_df.shape)
label_per_gesture_df.head()

(2500,)


0      pan
64     pan
128    pan
192    pan
256    pan
Name: Gesture_ID, dtype: object

In [14]:
full_pca40_train_XY_df[metadata_cols].iloc[::64].head()

,Participant,Gesture_ID,Gesture_Num
0,P102,pan,1
64,P102,pan,2
128,P102,pan,3
192,P102,pan,4
256,P102,pan,5


In [15]:
# Convert the DataFrame column to numpy array
labels_array = label_per_gesture_df.to_numpy()
# Create OneHotEncoder instance
gesture_encoder = OneHotEncoder(sparse=False)
# Reshape the labels array to fit the OneHotEncoder input shape
labels_reshaped = labels_array.reshape(-1, 1)
# Encode the labels
labels_encoded = gesture_encoder.fit_transform(labels_reshaped)
# Convert the encoded labels back to DataFrame
encoded_labels_df = pd.DataFrame(labels_encoded, columns=gesture_encoder.categories_[0])
print(encoded_labels_df.shape)
encoded_labels_df.head()

(2500, 10)


,close,delete,duplicate,move,open,pan,rotate,select-single,zoom-in,zoom-out
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
label_per_test_gesture_df = full_pca40_gesture_test_labels_df.iloc[::64]
gesture_encoder = OneHotEncoder(sparse=False)
test_labels_encoded = gesture_encoder.fit_transform(label_per_test_gesture_df.to_numpy().reshape(-1, 1))
encoded_test_labels_df = pd.DataFrame(test_labels_encoded, columns=gesture_encoder.categories_[0])

In [17]:
y_integer = np.argmax(encoded_labels_df, axis=1)
print(len(y_integer))

ytest_integer = np.argmax(encoded_test_labels_df, axis=1)
print(len(ytest_integer))

2500
700


Clustering in Input Space

In [18]:
X_2Dinput_PCA40 = X_3DTensor_PCA40.reshape(X_3DTensor_PCA40.shape[0], -1).detach().numpy()
Xtest_2Dinput_PCA40 = Xtest_3DTensor_PCA40.reshape(Xtest_3DTensor_PCA40.shape[0], -1).detach().numpy()

print(f"X_2Dinput_PCA40 shape: {X_2Dinput_PCA40.shape}")
print(f"Xtest_2Dinput_PCA40 shape: {Xtest_2Dinput_PCA40.shape}")

X_2Dinput_PCA40 shape: (2500, 5632)
Xtest_2Dinput_PCA40 shape: (700, 5632)


## KNN
Training knn, just using the closest neighbor (k=1)

In [19]:
# Define and train the KNN classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_2Dinput_PCA40, encoded_labels_df)

# Predict on the test data
y_pred = knn1.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(encoded_test_labels_df, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 23.00%


Training knn, k=10

In [20]:
# Define and train the KNN classifier
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_2Dinput_PCA40, encoded_labels_df)

# Predict on the test data
y_pred = knn10.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(encoded_test_labels_df, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 11.86%


Trying but no longer one-hot encoded

k=1

In [21]:
# Define and train the KNN classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_2Dinput_PCA40, y_integer)

# Predict on the test data
y_pred = knn1.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(ytest_integer, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 23.00%


k=10

In [22]:
# Define and train the KNN classifier
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_2Dinput_PCA40, y_integer)

# Predict on the test data
y_pred = knn10.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(ytest_integer, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 21.00%


## LSTM

In [23]:
batch_size = 32

training_dataset = GestureDatasetClustering(X_3DTensor_PCA40, y_integer)
nn_train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

testing_dataset = GestureDatasetClustering(Xtest_3DTensor_PCA40, ytest_integer)
nn_test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [24]:
# Model parameters
input_size = X_3DTensor_PCA40.shape[-1]
hidden_size = 128  # Number of features in the hidden state
num_layers = 2  # Number of recurrent layers
num_classes = len(unique_gestures)
num_epochs = 15
lr = 0.001

# Loss and optimizer
model = GestureLSTMClassifier_Hidden(input_size, hidden_size, num_layers, num_classes)

print(model)

GestureLSTMClassifier_Hidden(
  (lstm): LSTM(88, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


In [25]:
model_rawEMG, cls_report_rawEMG, labels_lst_rawEMG = train_LSTM_gesture_classifier(model, nn_train_loader, nn_test_loader, lr=lr, num_epochs=num_epochs, batch_size=32, use_hidden=True)

Starting training!
Epoch [1/15], Train Loss: 2.1570, Train Accuracy: 20.59%, Val Loss: 2.3882, Val Accuracy: 18.01%
Epoch [2/15], Train Loss: 1.5498, Train Accuracy: 47.00%, Val Loss: 2.6883, Val Accuracy: 22.02%
Epoch [3/15], Train Loss: 1.0424, Train Accuracy: 65.42%, Val Loss: 2.8646, Val Accuracy: 22.02%
Epoch [4/15], Train Loss: 0.7344, Train Accuracy: 75.08%, Val Loss: 3.2077, Val Accuracy: 19.49%
Epoch [5/15], Train Loss: 0.5264, Train Accuracy: 82.49%, Val Loss: 3.4409, Val Accuracy: 20.54%
Epoch [6/15], Train Loss: 0.3594, Train Accuracy: 88.38%, Val Loss: 3.7487, Val Accuracy: 20.68%
Epoch [7/15], Train Loss: 0.2279, Train Accuracy: 92.67%, Val Loss: 3.9041, Val Accuracy: 23.96%
Epoch [8/15], Train Loss: 0.1993, Train Accuracy: 93.83%, Val Loss: 3.9750, Val Accuracy: 23.36%
Epoch [9/15], Train Loss: 0.1846, Train Accuracy: 94.23%, Val Loss: 4.0910, Val Accuracy: 22.62%
Epoch [10/15], Train Loss: 0.1537, Train Accuracy: 95.59%, Val Loss: 4.2403, Val Accuracy: 19.49%
Epoch [11/

# EMG Only

In [26]:
# Kai's laptop
#data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\PCA_40D\\"
#data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\Filtered_Datasets\\"
#model_dir_path = 'C:\\Users\\kdmen\\Desktop\\Research\\Repos\\fl-gestures\\models\\Embedding\\Autoencoders\\'

# BRC Desktop
data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\EMG_PPD\\"
full_data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\Filtered_Datasets\\"
embedding_data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\Autoencoders\\EMG_Only\\"
model_dir_path = 'C:\\Users\\YamagamiLab\\Desktop\\Dev\\fl-gestures\\models\\Embedding\\Autoencoders\\'

print("Loading")

metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']

emg_full_train_XY_df = pd.read_pickle(data_path+'training_users_df.pkl')
emg_full_test_XY_df = pd.read_pickle(data_path+'test_users_df.pkl')
emg_full_gesture_train_labels_df = emg_full_train_XY_df['Gesture_ID']
emg_full_gesture_test_labels_df = emg_full_test_XY_df['Gesture_ID']
emg_full_user_train_labels_df = emg_full_train_XY_df['Participant']
emg_full_user_test_labels_df = emg_full_test_XY_df['Participant']
emg_full_training_u_df = emg_full_train_XY_df.drop(metadata_cols, axis=1)
emg_full_test_users_df = emg_full_test_XY_df.drop(metadata_cols, axis=1)


Loading


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Kai_MetaGestureClustering_24\\saved_datasets\\EMG_PPD\\training_users_df.pkl'

In [ ]:
# Get list of all the unique gesture names
all_train_test_gestures_df = pd.concat([full_pca40_gesture_train_labels_df, full_pca40_gesture_test_labels_df], axis=0)
unique_gestures = all_train_test_gestures_df.unique()

# Create OneHotEncoder instance
gesture_encoder = OneHotEncoder(sparse=False)
gesture_labels_reshaped = unique_gestures.reshape(-1, 1)
gesture_encoded = gesture_encoder.fit_transform(gesture_labels_reshaped)

In [ ]:
# CREATE THE TRAINING SET
num_rows_per_gesture = 64 # From the interp
num_gestures = len(full_pca40_training_u_df) // num_rows_per_gesture
num_features = full_pca40_training_u_df.shape[1]

# Ensure the data can be evenly divided into gestures
assert len(full_pca40_training_u_df) % num_rows_per_gesture == 0, "The total number of rows is not a multiple of the number of rows per gesture."

# Reshape into (batch_dim, time_step, n_features) AKA (n_gestures, n_rows_per_gesture, n_columns)
X_3D_PCA40 = full_pca40_training_u_df.to_numpy().reshape(num_gestures, num_rows_per_gesture, num_features)
#flattened_PCA = PCA_np.reshape(num_gestures, -1)

# Convert to PyTorch tensor
X_3DTensor_PCA40 = torch.tensor(X_3D_PCA40, dtype=torch.float32)

#################################################

# CREATE THE TEST SET
num_test_gestures = len(full_pca40_test_users_df) // num_rows_per_gesture
# Ensure the data can be evenly divided into gestures
assert len(full_pca40_test_users_df) % num_rows_per_gesture == 0, "The total number of rows is not a multiple of the number of rows per gesture."

# Reshape into (batch_dim, time_step, n_features) AKA (n_gestures, n_rows_per_gesture, n_columns) and convert to torch tensor
## Theres probably an easier way to just create it as a torch tensor lol
Xtest_3DTensor_PCA40 = torch.tensor(full_pca40_test_users_df.to_numpy().reshape(num_test_gestures, num_rows_per_gesture, num_features), dtype=torch.float32)


In [ ]:
label_per_gesture_df = full_pca40_gesture_train_labels_df.iloc[::64]
print(label_per_gesture_df.shape)
label_per_gesture_df.head()

In [ ]:
# Convert the DataFrame column to numpy array
labels_array = label_per_gesture_df.to_numpy()
# Create OneHotEncoder instance
gesture_encoder = OneHotEncoder(sparse=False)
# Reshape the labels array to fit the OneHotEncoder input shape
labels_reshaped = labels_array.reshape(-1, 1)
# Encode the labels
labels_encoded = gesture_encoder.fit_transform(labels_reshaped)
# Convert the encoded labels back to DataFrame
encoded_labels_df = pd.DataFrame(labels_encoded, columns=gesture_encoder.categories_[0])
print(encoded_labels_df.shape)
encoded_labels_df.head()

In [ ]:
label_per_test_gesture_df = full_pca40_gesture_test_labels_df.iloc[::64]
gesture_encoder = OneHotEncoder(sparse=False)
test_labels_encoded = gesture_encoder.fit_transform(label_per_test_gesture_df.to_numpy().reshape(-1, 1))
encoded_test_labels_df = pd.DataFrame(test_labels_encoded, columns=gesture_encoder.categories_[0])

In [ ]:
y_integer = np.argmax(encoded_labels_df, axis=1)
print(len(y_integer))

ytest_integer = np.argmax(encoded_test_labels_df, axis=1)
print(len(ytest_integer))

In [ ]:
X_2Dinput_PCA40 = X_3DTensor_PCA40.reshape(X_3DTensor_PCA40.shape[0], -1).detach().numpy()
Xtest_2Dinput_PCA40 = Xtest_3DTensor_PCA40.reshape(Xtest_3DTensor_PCA40.shape[0], -1).detach().numpy()

print(f"X_2Dinput_PCA40 shape: {X_2Dinput_PCA40.shape}")
print(f"Xtest_2Dinput_PCA40 shape: {Xtest_2Dinput_PCA40.shape}")

In [ ]:
# Define and train the KNN classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_2Dinput_PCA40, encoded_labels_df)

# Predict on the test data
y_pred = knn1.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(encoded_test_labels_df, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

In [ ]:
# Define and train the KNN classifier
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_2Dinput_PCA40, encoded_labels_df)

# Predict on the test data
y_pred = knn10.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(encoded_test_labels_df, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

In [ ]:
# Define and train the KNN classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_2Dinput_PCA40, y_integer)

# Predict on the test data
y_pred = knn1.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(ytest_integer, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

In [ ]:
# Define and train the KNN classifier
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_2Dinput_PCA40, y_integer)

# Predict on the test data
y_pred = knn10.predict(Xtest_2Dinput_PCA40)

# Calculate accuracy
accuracy = accuracy_score(ytest_integer, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

In [ ]:
batch_size = 32

training_dataset = GestureDatasetClustering(X_3DTensor_PCA40, y_integer)
nn_train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

testing_dataset = GestureDatasetClustering(Xtest_3DTensor_PCA40, ytest_integer)
nn_test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
# Model parameters
input_size = X_3DTensor_PCA40.shape[-1]
hidden_size = 128  # Number of features in the hidden state
num_layers = 2  # Number of recurrent layers
num_classes = len(unique_gestures)
num_epochs = 15
lr = 0.001

# Loss and optimizer
model = GestureLSTMClassifier_Hidden(input_size, hidden_size, num_layers, num_classes)

print(model)

In [ ]:
model_rawEMG, cls_report_rawEMG, labels_lst_rawEMG = train_LSTM_gesture_classifier(model, nn_train_loader, nn_test_loader, lr=lr, num_epochs=num_epochs, batch_size=32, use_hidden=True)